# IMPORT

In [1]:
import torch

import sys
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/") # PATH TO PYSEQM

import seqm
from seqm.seqm_functions.constants import Constants
from seqm.seqm_functions.make_dm_guess import make_dm_guess
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from seqm.seqm_functions.read_xyz import read_xyz

from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import os
import numpy as np

seqm.seqm_functions.scf_loop.debug=True
seqm.seqm_functions.scf_loop.MAX_ITER=700

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Ti-Fe

In [2]:
seqm.seqm_functions.scf_loop.MAX_ITER=2000


species, coordinates = read_xyz(['dimers_.xyz', 'dimers_stretched.xyz', 
                                ])
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]



const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'PM6',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-5,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0,0.9], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : False,
                    "Hf_flag": False,
                    'scf_backward' : 0,
                    'UHF' : False,
                   }

#charges = torch.tensor([0],dtype=torch.int64, device=device)
molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)
esdriver = Electronic_Structure(seqm_parameters).to(device)
esdriver(molecules, 
         #P0=dm
        )

/home/maxim/Projects/git2/PYSEQM_dev/seqm/seqm_functions/diag_d.py:109: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2794.)
  e0,v = torch.symeig(x0,eigenvectors=True,upper=True)


scf  0 42.33941968152385 2 0.44315361976623535
scf  1 31.486071256013474 2 0.032221078872680664
scf  2 23.021773687621135 2 0.03226470947265625
scf  3 16.46685758009403 2 0.031911373138427734
scf  4 11.511916416621261 2 0.032241106033325195
scf  5 9.574081992663196 2 0.031896114349365234
scf  6 7.965387116131296 2 0.03251147270202637
scf  7 8.052293345740736 2 0.03938436508178711
scf  8 7.638405155184046 2 0.040631771087646484
scf  9 7.039127005608634 2 0.040581464767456055
scf  10 6.20735565048426 2 0.04059648513793945
scf  11 6.065070285431148 2 0.032029151916503906
scf  12 5.421035603298606 2 0.031497955322265625
scf  13 5.127369048844685 2 0.031897544860839844
scf  14 4.654122001887572 2 0.03177070617675781
scf  15 4.3401306794094126 2 0.031888484954833984
scf  16 3.974952435234627 2 0.031980037689208984
scf  17 3.673350106570524 2 0.0319674015045166
scf  18 3.388623555051481 2 0.03176069259643555
scf  19 3.126607078778079 2 0.031969308853149414
scf  20 2.9032477478235705 2 0.03172

In [3]:
#dm = molecules.dm.clone()

In [4]:
# Absolute energies
print('Eelec:',molecules.Eelec[0].item(),molecules.Eelec[1].item())
# MOPAC: -1563.9357  -1598.8704
# NEXMD: -1569.54353790   -1604.52140895

Eelec: -1563.9200883525834 -1598.758905922045


In [5]:
# Energy differences
print('Eelec:', 'PYSEQM', molecules.Eelec[0].item()-molecules.Eelec[1].item(),  'MOPAC', -1563.9357  +1598.8704, 'NEXMD', -1569.54353790   +1604.52140895)
# MOPAC: -1563.9357  -1598.8704
# NEXMD: -1569.54353790 

Eelec: PYSEQM 34.838817569461526 MOPAC 34.93470000000002 NEXMD 34.977871049999976
